### Used Libraries<a class="anchor" id="chapter1"></a>

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Access to the DB <a class="anchor" id="chapter2"></a>

In [2]:
db_config = {'user': 'practicum_student',                         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs',                   # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,                                        # connection port
             'db': 'data-analyst-fc-barcelona-20-21-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


## The Database

### squad table:
**player_id:** player's id<br>

**first_name:** player's first name<br>

**last_name:** player's last name<br>

**position:** position in the field - gk,df,mf,fw<br>


### games_played table:
**player_id:** player's id

**league:** league appearances

**champions:** champions appearances

**cup:** cup appearances

**supercup:** supercup appearances

### league_stats table:
**player_id:** player's id

**league_goals:** goals in league

**league_assits:** assits in league

**league_yellow:** yellow cards in league

**league_red:** red cards in league

**league_minutes:** minutes played in league

### champions_stats table:
**player_id:** player's id

**champions_goals:** goals in champions

**champions_assits:** assits in champions

**champions_yellow:** yellow cards in champions

**champions_red:** red cards in champions

**champions_minutes:** minutes played in champions


### cup_stats table:
**player_id:** player's id

**cup_goals:** goals in cup

**cup_assits:** assits in cup

**cup_yellow:** yellow cards in cup

**cup_red:** red cards in cup

**cup_minutes:** minutes played in cup

### supercup_stats table:
**player_id:** player's id

**supercup_goals:** goals in supercup

**supercup_assits:** assits in supercup

**supercup_yellow:** yellow cards in supercup

**supercup_red:** red cards in supercup

**supercup_minutes:** minutes played in supercup



#  Table Queries <a class="anchor" id="chapter3"></a>

function that takes a query and return dataframe for general use

In [3]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

### 1. What is the position played by the majority of the squad?

In [4]:
query = '''
        SELECT
            position,
            COUNT(DISTINCT player_id) AS position_cnt
        FROM
            squad
        GROUP BY
            position
        ORDER BY
            position_cnt DESC;
        '''
print()
print('The number of players in each position:')
queryResult(query)


The number of players in each position:


,position,position_cnt
0,df,9
1,fw,8
2,mf,8
3,gk,4


**df** has the most players

----

### 2. Which players played the most games during the entire season?

In [5]:
query = '''
        SELECT
            games_played.player_id,
            squad.first_name,
            squad.last_name,
            games_played.league+
            games_played.champions+
            games_played.cup+
            games_played.supercup AS total_games
        FROM
            games_played
            LEFT JOIN squad ON squad.player_id = games_played.player_id
        ORDER BY
            total_games DESC;
            
        '''
print()
print('The players who played most games:')
queryResult(query)


The players who played most games:


,player_id,first_name,last_name,total_games
0,19,Pedro,Lopez,52
1,28,Antoine,Griezmann,51
2,15,Frenkie,De Jong,51
3,14,Sergio,Busquets,50
4,12,Jordi,Alba,49
5,29,Lionel,Messi,48
6,9,Clément,Lenglet,48
7,24,Ousmane,Dembélé,44
8,1,Marc-André,Ter Stegen,42
9,25,Francisco,Trincão,42


**Player 19 Pedro Lopez** played the most games, followed by 28, 15, 14, 12, 29, 9.

----

### 3. Out of the players who played less than 1000 minutes in the league who scored the most goals?

In [26]:
query = '''
        SELECT
            squad.first_name,
            squad.last_name,
            league_stats.league_goals
            
        FROM
            league_stats
            LEFT JOIN squad ON squad.player_id = league_stats.player_id
            
        WHERE
            league_stats.league_minutes < 1000 
            
        ORDER BY
            league_stats.league_goals DESC;
        '''
print()
print('The playes with most goals under 1,000 minutes game play:')
queryResult(query)


The playes with most goals under 1,000 minutes game play:


,first_name,last_name,league_goals
0,Ansu,Fati,4
1,Francisco,Trincão,3
2,Philippe,Coutinho,2
3,Ilaix,Moriba,1
4,Sergi,Roberto,1
5,Junior,Firpo,1
6,Riqui,Puig,1
7,Norberto,Murara,0
8,Álex,Collado,0
9,Iñaki,Peña,0


**Player 22 Ansu Fati** has the most goals under 1,000 minutes game play.

----

### 4. Which players played the most minutes in the league? Show top 5.

In [7]:
query = '''
        SELECT
            league_stats.player_id,
            squad.first_name,
            squad.last_name,
            league_stats.league_minutes
        FROM
            league_stats
            LEFT JOIN squad ON squad.player_id = league_stats.player_id
        ORDER BY
            league_minutes DESC
        LIMIT 5;    
        '''
print()
print('The players who played the most time:')
queryResult(query)


The players who played the most time:


,player_id,first_name,last_name,league_minutes
0,15,Frenkie,De Jong,3158
1,12,Jordi,Alba,3030
2,29,Lionel,Messi,3022
3,1,Marc-André,Ter Stegen,2790
4,28,Antoine,Griezmann,2619


----

### 5.How many assists were made by the forwards?

In [8]:
query = '''
        SELECT
            
            squad.position,
            SUM(league_stats.league_assists)+
            SUM(champions_stats.champions_assists)+
            SUM(cup_stats.cup_assists)+
            SUM(supercup_stats.supercup_assists) AS total_assists
        FROM
            league_stats
            LEFT JOIN squad ON squad.player_id = league_stats.player_id
            LEFT JOIN champions_stats ON champions_stats.player_id = league_stats.player_id
            LEFT JOIN cup_stats ON cup_stats.player_id = league_stats.player_id
            LEFT JOIN supercup_stats ON supercup_stats.player_id = league_stats.player_id
        WHERE
            squad.position = 'fw'
        GROUP BY
            squad.position
        ORDER BY
            total_assists DESC;
        '''
print()
print('The number of asists per position:')
queryResult(query)


The number of asists per position:


,position,total_assists
0,fw,42


**42 assits** by forwards.

----

### 6. Who are the top 3 players with the best goals per minute record?

In [22]:
query = '''
            SELECT
                squad.first_name,
                squad.last_name,
                league_stats.player_id,
                
                 
                ((league_stats.league_minutes + champions_stats.champions_minutes+
                cup_stats.cup_minutes + supercup_stats.supercup_minutes ) 
                /
                (league_stats.league_goals + champions_stats.champions_goals+
                cup_stats.cup_goals + supercup_stats.supercup_goals)) AS minutes_per_goal
                
            FROM
                league_stats
                LEFT JOIN squad ON squad.player_id = league_stats.player_id
                LEFT JOIN champions_stats ON champions_stats.player_id = league_stats.player_id
                LEFT JOIN cup_stats ON cup_stats.player_id = league_stats.player_id
                LEFT JOIN supercup_stats ON supercup_stats.player_id = league_stats.player_id
            
            WHERE
                (league_goals + champions_goals + cup_goals + supercup_goals) > 0
            
            ORDER BY
                minutes_per_goal
                
            LIMIT 3;
        '''
print()
print('The players with the highest goal per minute count:')
queryResult(query)


The players with the highest goal per minute count:


,first_name,last_name,player_id,minutes_per_goal
0,Lionel,Messi,29,110
1,Ansu,Fati,22,119
2,Antoine,Griezmann,28,195


----

### 7. How many players did not contribute goals or assists in the league?

In [10]:
query = '''
        SELECT
            COUNT(league_stats.player_id) AS non_contributing_players
        FROM
            league_stats
            LEFT JOIN squad ON squad.player_id = league_stats.player_id
        WHERE
            league_stats.league_goals = 0
            AND
            league_stats.league_assists = 0
        '''
print()
print('The number of players who did not contribute goals or assists:')
queryResult(query)


The number of players who did not contribute goals or assists:


,non_contributing_players
0,10


----

### 8. Who is the player that has the most yellow cards in the league? (If there is a tie then sort by minutes played)

In [11]:
query = '''
        SELECT
            league_stats.player_id,
            squad.first_name,
            squad.last_name,
            league_stats.league_yellow,
            league_stats.league_minutes
        FROM
            league_stats
            LEFT JOIN squad ON squad.player_id = league_stats.player_id 
        ORDER BY
            league_stats.league_yellow DESC,
            league_stats.league_minutes
        LIMIT 3;   
        '''
print()
print('The players with most yellow cards:')
queryResult(query)


The players with most yellow cards:


,player_id,first_name,last_name,league_yellow,league_minutes
0,14,Sergio,Busquets,9,2527
1,12,Jordi,Alba,9,3030
2,9,Clément,Lenglet,7,2476


**Player 14 Sergio Busquets** has the most yellow cards per minute played.

----

### 9. Which defender contribute the most goals and assists in the league?

In [12]:
query = '''
        SELECT
            league_stats.player_id,
            squad.first_name,
            squad.last_name,
            squad.position,
            league_stats.league_goals,
            league_stats.league_assists
        FROM
            league_stats
            LEFT JOIN squad ON squad.player_id = league_stats.player_id 
        WHERE
            squad.position = 'df'
        ORDER BY
            league_stats.league_goals DESC,
            league_stats.league_assists DESC
        LIMIT 5;  
        '''
print()
print('The defence players with most assists and goals in the league:')
queryResult(query)


The defence players with most assists and goals in the league:


,player_id,first_name,last_name,position,league_goals,league_assists
0,12,Jordi,Alba,df,3,7
1,11,Óscar,Mingueza,df,2,2
2,6,Sergiño,Dest,df,2,1
3,8,Ronald,Araújo,df,2,1
4,13,Sergi,Roberto,df,1,2


**Player 12 Jordi Alba** has the most goals and assists from defence players in the league.

----

### 10. How many players played more then 50 hours total in the league and in the champions league?

In [30]:
query = '''
            SELECT
                league_stats.player_id,
                squad.first_name,
                squad.last_name,
                ((league_stats.league_minutes)+(champions_stats.champions_minutes)) / 60 AS hours_played 
            
            FROM
                league_stats
                LEFT JOIN squad ON squad.player_id = league_stats.player_id
                LEFT JOIN champions_stats ON champions_stats.player_id = league_stats.player_id
            
            WHERE
            ((league_stats.league_minutes)+(champions_stats.champions_minutes)) / 60 > 50

            ORDER BY
                hours_played DESC;
        '''
print()
print('The players with more than 50 hours played on league and champions:')
queryResult(query)


The players with more than 50 hours played on league and champions:


,player_id,first_name,last_name,hours_played
0,15,Frenkie,De Jong,62
1,29,Lionel,Messi,59
2,12,Jordi,Alba,58
3,1,Marc-André,Ter Stegen,54
4,28,Antoine,Griezmann,52
